In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from scipy import stats
from keras import backend as K
from keras.callbacks import TensorBoard, EarlyStopping
from sklearn.model_selection import StratifiedKFold
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.layers import Activation
from keras.layers import MaxPooling1D, UpSampling1D
from keras.layers import MaxPooling2D
from keras.layers import Flatten, Reshape
from keras.layers import Input, Dense, Bidirectional
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import Conv2D
from keras.layers import RepeatVector
from keras.layers.pooling import GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Model
from keras.models import Sequential
import numpy as np
import os
import matplotlib.pyplot as plt
import random as rand
import pandas as pd
import tensorflow
import keras
import statistics as st
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils.vis_utils import plot_model
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import p_f1_recall as metric

In [ ]:
results = []

In [ ]:
### HYPERPARAMETERS ####
batch_size, epochs, verbose = 200, 100, 1
group_size = 48
encoding_dim = 8

In [ ]:
##### First proces unlabled data

data_u = pd.read_csv('fullest_unlabeled_data.csv')

### Remove unncessary columns
data_u = data_u.iloc[:,[3,4,5,7,8,9,10]]

### Standardize the features
def standardize(data,col_nums_to_standardize):
    for col in col_nums_to_standardize:
        data.iloc[:,col] = (data.iloc[:,col] - np.mean(data.iloc[:,col]))/np.std(data.iloc[:,col])
    return data

data_u = standardize(data_u,col_nums_to_standardize = range(6))


#### Now process the labeled data
data_train = pd.read_csv('final_training.csv')
data_test = pd.read_csv('final_testing.csv')

### Remove unncessary columns
data_train = data_train.iloc[:,[3,4,5,7,8,9,10,12]]  ## Add 13 if need to keep IDs ## Drop heading because of low mutual information
data_test = data_test.iloc[:,[3,4,5,7,8,9,10,12]]

### Standardize the features
data_train = standardize(data_train,col_nums_to_standardize = range(6))
data_test = standardize(data_test,col_nums_to_standardize = range(6))

In [ ]:
#Break features into groups of size n

def break_into_samples(data,group_size):
    l = []
    sep_ts = set(data.loc[:,'ID'])
    for ts in sep_ts:
        sub_data  = np.array(data.loc[data.loc[:,'ID']== ts,:])
        a = int(sub_data.shape[0]/group_size)
        if a != 0:
            sub_data = sub_data[:(a*group_size),:]
            sub_data = [np.split(sub_data,a)]
            l = l + sub_data
    data = np.vstack(l)
    data = data[:,:,:-1]
    return data

chunked_data_train = break_into_samples(data_train,group_size)
chunked_data_test = break_into_samples(data_test,group_size)
chunked_u_data = break_into_samples(data_u,group_size)

In [ ]:
def split_train_val(data,labeled = True,thresh = 0.8):
    np.random.shuffle(data)
    thresh1 = int(data.shape[0]*thresh)
    training = data[:thresh1,:,:]
    val = data[thresh1:,:,:]
    return training, val


global_num_classes = -1

def split_feat_lab(data,many_to_one = True, prop = 0.65):
    features = data[:,:,:data.shape[2]-1]
    labels = data[:,:,data.shape[2]-1]
    if many_to_one == True:
        y_temp = np.zeros(labels.shape[0])
        y_temp.fill(np.nan)
        for i in range(labels.shape[0]):
            row = labels[i,:]
            trans_class = (Counter(row)).most_common(1)[0]
            if trans_class[1] < group_size*prop : 
                 y_temp[i] = 4
            else:
                 y_temp[i] = st.mode(row)
        labels = y_temp
        global global_num_classes
    if global_num_classes < 0:
        global_num_classes = len(set(labels))
    num_classes = global_num_classes
    labels = to_categorical(labels,num_classes)
    return features,labels

In [ ]:
chunked_data_train1, chunked_data_val = split_train_val(chunked_data_train)
training, val = split_train_val(chunked_u_data)

train_x,train_y = split_feat_lab(chunked_data_train1)
val_x,val_y = split_feat_lab(chunked_data_val)
test_x,test_y = split_feat_lab(chunked_data_test)

In [ ]:
train_x_flat = train_x.reshape(train_x.shape[0],(train_x.shape[1] * train_x.shape[2]))
val_x_flat = val_x.reshape(val_x.shape[0],(val_x.shape[1] * val_x.shape[2]))
test_x_flat = test_x.reshape(test_x.shape[0],(test_x.shape[1] * test_x.shape[2]))

training_flat = training.reshape(training.shape[0],(training.shape[1]*training.shape[2]))
val_flat = val.reshape(val.shape[0],(val.shape[1]*val.shape[2]))

In [ ]:

def build_autoencoder(encoding_dim = encoding_dim): 
    raw_input = Input(shape=(training_flat.shape[1],))
    encoding_1 = Dense(encoding_dim*4, activation='relu')(raw_input)
    encoding_2 = Dense(encoding_dim*2, activation='relu')(encoding_1)
    encoded = Dense(encoding_dim, activation='relu')(encoding_2)
    decoding_1 = Dense(encoding_dim*2, activation='relu')(encoded)
    decoding_2 = Dense(encoding_dim*4, activation='relu')(decoding_1)
    decoded = Dense(training_flat.shape[1])(decoding_2)
    autoencoder = Model(raw_input, decoded)
    encoder = Model(raw_input, encoded)
    return autoencoder, encoder
    
autoencoder, encoder = build_autoencoder()
init_weights = encoder.get_weights() ## Saves initial weights for use later

def autoencoder_train(autoencoder):
    autoencoder.compile(optimizer='adam', loss='mse')
    autoencoder.fit(training_flat, training_flat,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(val_flat, val_flat),
                callbacks=[EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto', baseline=None, restore_best_weights=True)])
    loss = autoencoder.evaluate(val_flat, val_flat, batch_size=batch_size, verbose=verbose)
    return loss

autoencoder_loss = autoencoder_train(autoencoder)
autoencoder.summary()

trained_weights = encoder.get_weights() ### This allows us to fine tune below, 
#but then initialize to these weights if we want to fine tune in a different way without retraining our autoencoder

Train on 116447 samples, validate on 29112 samples
Epoch 1/100
116447/116447 [==============================] - 3s 28us/step - loss: 0.3134 - val_loss: 0.1445
Epoch 2/100
116447/116447 [==============================] - 2s 18us/step - loss: 0.1377 - val_loss: 0.1374
Epoch 3/100
116447/116447 [==============================] - 2s 17us/step - loss: 0.1340 - val_loss: 0.1349
Epoch 4/100
116447/116447 [==============================] - 2s 18us/step - loss: 0.1326 - val_loss: 0.1341
Epoch 5/100
116447/116447 [==============================] - 2s 17us/step - loss: 0.1318 - val_loss: 0.1338
Epoch 6/100
116447/116447 [==============================] - 2s 17us/step - loss: 0.1241 - val_loss: 0.1202
Epoch 7/100
116447/116447 [==============================] - 2s 17us/step - loss: 0.1185 - val_loss: 0.1187
Epoch 8/100
116447/116447 [==============================] - 2s 17us/step - loss: 0.1178 - val_loss: 0.1184
Epoch 9/100
116447/116447 [==============================] - ETA: 0s - loss: 0.117 - 

In [ ]:
encoder.set_weights(trained_weights)
new_layer = Dense(global_num_classes, activation='softmax')(encoder.output)
pre_trained = Model(encoder.input,new_layer)
pre_trained.summary()

def train_model(model):
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',metric.recall,metric.precision,metric.f1])
    model.fit(train_x_flat,train_y,epochs=epochs,batch_size=batch_size,verbose=verbose,validation_data=(val_x_flat,val_y),
             callbacks=[EarlyStopping(monitor='val_f1', min_delta=0, patience=7, verbose=0, mode='max', baseline=None, restore_best_weights=True)])
    loss,accuracy,recall_score,precision_score,f_score = model.evaluate(val_x_flat, val_y, batch_size=batch_size, verbose=verbose)
    return loss,accuracy,recall_score,precision_score,f_score

m1_loss,m1_accuracy,m1_recall_score,m1_precision_score,m1_f_score = train_model(pre_trained)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 288)               0         
_________________________________________________________________
dense_50 (Dense)             (None, 32)                9248      
_________________________________________________________________
dense_51 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_52 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_56 (Dense)             (None, 5)                 45        
Total params: 9,957
Trainable params: 9,957
Non-trainable params: 0
_________________________________________________________________
Train on 969 samples, validate on 243 samples
Epoch 1/100
969/969 [==============================] - 1s 884us/step - loss: 5.1492 - acc: 0.2

Epoch 30/100
969/969 [==============================] - 0s 20us/step - loss: 0.7601 - acc: 0.7172 - recall: 0.6171 - precision: 0.7921 - f1: 0.6936 - val_loss: 0.7965 - val_acc: 0.7202 - val_recall: 0.6091 - val_precision: 0.7993 - val_f1: 0.6910
Epoch 31/100
969/969 [==============================] - 0s 15us/step - loss: 0.7542 - acc: 0.7172 - recall: 0.6192 - precision: 0.7821 - f1: 0.6906 - val_loss: 0.7854 - val_acc: 0.7243 - val_recall: 0.6091 - val_precision: 0.8081 - val_f1: 0.6942
Epoch 32/100
243/243 [==============================] - 0s 12us/step


In [ ]:
encoder.set_weights(init_weights) ### Reinitializes the encoder
new_layer = Dense(global_num_classes, activation='softmax')(encoder.output)
no_pre_training = Model(encoder.input,new_layer)  ## This just makes a model without the encoding

m2_loss,m2_accuracy,m2_recall_score,m2_precision_score,m2_f_score= train_model(no_pre_training)

Train on 969 samples, validate on 243 samples
Epoch 1/100
969/969 [==============================] - 1s 932us/step - loss: 1.6010 - acc: 0.2270 - recall: 0.0000e+00 - precision: 0.0000e+00 - f1: 0.0000e+00 - val_loss: 1.5026 - val_acc: 0.4280 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_f1: 0.0000e+00
Epoch 2/100
969/969 [==============================] - 0s 17us/step - loss: 1.4790 - acc: 0.4272 - recall: 0.0010 - precision: 0.1744 - f1: 0.0021 - val_loss: 1.4116 - val_acc: 0.5267 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_f1: 0.0000e+00
Epoch 3/100
969/969 [==============================] - 0s 21us/step - loss: 1.3889 - acc: 0.5088 - recall: 0.0134 - precision: 0.7413 - f1: 0.0259 - val_loss: 1.3210 - val_acc: 0.5638 - val_recall: 0.0247 - val_precision: 0.6173 - val_f1: 0.0475
Epoch 4/100
969/969 [==============================] - 0s 24us/step - loss: 1.3032 - acc: 0.5666 - recall: 0.1011 - precision: 0.8066 - f1: 0.1754 - val_loss: 1.2396 - val_acc: 0.

Epoch 34/100
969/969 [==============================] - 0s 19us/step - loss: 0.6692 - acc: 0.7534 - recall: 0.6543 - precision: 0.8359 - f1: 0.7335 - val_loss: 0.8230 - val_acc: 0.6914 - val_recall: 0.6049 - val_precision: 0.8259 - val_f1: 0.6981
Epoch 35/100
969/969 [==============================] - 0s 17us/step - loss: 0.6585 - acc: 0.7554 - recall: 0.6584 - precision: 0.8383 - f1: 0.7375 - val_loss: 0.8160 - val_acc: 0.6872 - val_recall: 0.5967 - val_precision: 0.8197 - val_f1: 0.6902
Epoch 36/100
969/969 [==============================] - 0s 17us/step - loss: 0.6514 - acc: 0.7626 - recall: 0.6646 - precision: 0.8365 - f1: 0.7401 - val_loss: 0.8219 - val_acc: 0.6872 - val_recall: 0.6049 - val_precision: 0.8124 - val_f1: 0.6932
Epoch 37/100
969/969 [==============================] - 0s 21us/step - loss: 0.6399 - acc: 0.7606 - recall: 0.6811 - precision: 0.8397 - f1: 0.7515 - val_loss: 0.8222 - val_acc: 0.6914 - val_recall: 0.6008 - val_precision: 0.7978 - val_f1: 0.6852
Epoch 38/100

In [ ]:
results.append([autoencoder_loss,m1_f_score,m2_f_score])

In [ ]:
results

[[0.14784393274840163, 0.14135833722328453, 0.5432675618203089],
 [0.14784393274840163, 0.2186464696754644, 0.6593394166648142],
 [0.11737923866061596, 0.6959811667355981, 0.6981268029644656]]